# Listing NSV content

## basic python setup

In [2]:
from pykg2tbl import DefaultSparqlBuilder, KGSource, QueryResult
from pathlib import Path
from pandas import DataFrame


# SPARQL EndPoint to use - wrapped as Knowledge-Graph 'source'
NSV_ENDPOINT: str = "https://vocab.nerc.ac.uk/sparql/sparql"
NSV:KGSource = KGSource.build(NSV_ENDPOINT)

TEMPLATES_FOLDER = str(Path().absolute())
GENERATOR = DefaultSparqlBuilder(templates_folder=TEMPLATES_FOLDER)


def generate_sparql(name: str, **vars) -> str: 
    """ Simply build the sparql by using the named query and applying the vars
    """
    return GENERATOR.build_syntax(name, **vars)


def execute_to_df(name: str, **vars) -> DataFrame:
    """ Builds the sparql and executes, returning the result as a dataframe.
    """
    sparql = generate_sparql(name, **vars)
    result: QueryResult = NSV.query(sparql=sparql)
    return result.to_dataframe()


## finding collections

The included sparql-template `./nsv-list-collections.sparql` allows for listing the collections that are available at the [NSV vocab server](https://vocab.nerc.ac.uk/)
You can call it:
* without any arguments to just get the full list
* optionally with arguments `N` (limit) and `O` (offset) to paginate the result
* optionally with argument `match` to narrow down to collections with a title matching your regex


In [8]:
# full list
execute_to_df("nsv-list-collections.sparql")

,Collection,Title,Description
0,A01,International Coastal Atlas Network Coastal Er...,Terms used at all hierarchical levels in the I...
1,A02,Oregon Coastal Atlas Coastal Erosion Thesaurus...,Terms used at all hierarchical levels in the O...
2,A03,Oregon Coastal Atlas Coastal Erosion Thesaurus...,Terms used at all hierarchical levels in the O...
3,A04,MIDA Coastal Erosion Thesaurus,A collection of terms used by the Irish Marine...
4,A05,AtlantOS Essential Variables,Collection of terms used to group key measurem...
...,...,...,...
289,W07,SensorML Identification Section Terms,Terms used in SensorML to identify an observat...
290,W08,SensorML Contact Section Terms,Terms used in SensorML to describe the role an...
291,W09,Sensor Web Enablement Observations and Measure...,Terms used in SWE Observations and Measurement...
292,W10,SensorML Output Section Data Interface Terms,Terms used in SensorML to describe the softwar...


In [9]:
# page 3 of size 20
size = 20
count = 3
execute_to_df("nsv-list-collections.sparql", N=size, O=(count -1)*size)

,Collection,Title,Description
0,C46,Marine Strategy Framework Directive criteria 2...,Concepts specified as criteria to be used to m...
1,C47,Marine Strategy Framework Directive indicators...,Concepts specified as indicators to be used to...
2,C48,MEDIN Data Guidelines,Documents produced by the Marine Environmental...
3,C59,BODC organisation roles within activities and ...,Generic terms used by BODC to describe the rel...
4,C60,NERC DataGrid vocabulary term relationships,Terms used in the NDG project to describe the ...
5,C61,BODC post town names,Labels used by BODC to populate 'city' or 'pos...
6,C62,BODC administrative region names,Labels used by BODC to populate 'county' metad...
7,C64,United Kingdom Charting Progress 2 sea regions,Terms used specify regions of the seas around ...
8,C67,BODC series parameter collection names,Terms used by BODC to describe groups of relat...
9,C71,BODC organisation categories,Terms used by BODC to classify the organisatio...


In [3]:
#matching for "platform"
term="platform"
execute_to_df("nsv-list-collections.sparql", regex=term)

,Collection,Title,Description
0,B76,BODC Platform Models,Terms used to describe designs or versions of ...
1,C17,ICES Platform Codes,Identifiers and metadata for platform instance...
2,EL1,SeaVoX Sampling and Observation Platform Event...,Terms used to identify actions performed on sa...
3,EL2,SeaVoX Sampling and Observation Platform Event...,Terms used to identify processes performed on ...
4,L06,SeaVoX Platform Categories,2-level grouping term hierarchy used for vehic...
5,P19,Global Change Master Directory platforms,Terms used to describe sensor-bearing platform...
6,R22,Argo platform family,List of platform family/category of Argo float...
7,R23,Argo platform type,List of Argo float types. Argo netCDF variable...
8,R24,Argo platform maker,List of Argo float manufacturers. Argo netCDF ...


## listing terms in selected collections 

The included sparql-template `./nsv-listing.sparql` allows for listing the terms in a selected collection at the [NSV vocab server](https://vocab.nerc.ac.uk/)
You can call it:
* with at least the `cc` argument indicating the code of the selected collection you want to list
* optionally with arguments `N` (limit) and `O` (offset) to paginate the result
* optionally with argument `lang` to get term labels in another language (defaults to 'en')

In [5]:
execute_to_df("nsv-listing.sparql", cc="P06")

,IDENTIFIER,PreferedLabelTranslated,AlternateLabel,Isdeprecated,ConceptIRI,AlternateLabelTranslated
0,SDN:P06::BQSM,Becquerels per square metre,Bq/m^2,false,http://vocab.nerc.ac.uk/collection/P06/current...,NaN
1,SDN:P06::BSM3,Becquerels second per cubic metre,Bq.s/m^3,false,http://vocab.nerc.ac.uk/collection/P06/current...,NaN
2,SDN:P06::KMP2,Kilograms per square metre,kg/m^2,false,http://vocab.nerc.ac.uk/collection/P06/current...,kg/m^2
3,SDN:P06::UKMC,Kilograms per cubic metre,kg/m^3,false,http://vocab.nerc.ac.uk/collection/P06/current...,kg/m^3
4,SDN:P06::UUUU,Dimensionless,Dmnless,false,http://vocab.nerc.ac.uk/collection/P06/current...,Dmnless
...,...,...,...,...,...,...
391,SDN:P06::UMIN,Minutes,min,false,http://vocab.nerc.ac.uk/collection/P06/current...,NaN
392,SDN:P06::SQCC,Square centimetres per cubic centimetre,cm^2/cm^3,false,http://vocab.nerc.ac.uk/collection/P06/current...,NaN
393,SDN:P06::MGAL,MilliGals,mGal,false,http://vocab.nerc.ac.uk/collection/P06/current...,NaN
394,SDN:P06::UNDX,Number per square metre per day,#/m^2/d,false,http://vocab.nerc.ac.uk/collection/P06/current...,NaN


## Searching for mathcing terms accross collections

The included sparql-template `./nsv-find.sparql` allows for finding a matching concept in a collection from the [NSV vocab server](https://vocab.nerc.ac.uk/)
You can call it:
* with at least the `collections` argument indicating a list of collections you want to search in - if '*' is passed the search goes across 
* optionally with argument `lang` to get term labels in another language (defaults to 'en')
* optionally with argument `regex` to pass the regex that should match the prefLabel of the Concept

In [9]:
term="Orca"
execute_to_df("nsv-find.sparql", regex=term, collections=['P01', 'S25'])

,ConceptIRI,Identifier,PreferedLabelTranslated
0,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::A97972A6,Count of Pseudorca crassidens (ITIS: 180463: W...
1,http://vocab.nerc.ac.uk/collection/P01/current...,SDN:P01::N9421078,Count of Orcinus orca (ITIS: 180469: WoRMS 137...
2,http://vocab.nerc.ac.uk/collection/S25/current...,SDN:S25::BE006521,Orcinus orca (ITIS: 180469: WoRMS 137102)
3,http://vocab.nerc.ac.uk/collection/S25/current...,SDN:S25::BE006523,Pseudorca crassidens (ITIS: 180463: WoRMS 137104)
